# Multi-class Logistic Regression with parameter search

This code demonstrates how to do a grid search for a regularization parameter C using multiclass logistic regression on the famous Iris dataset.

Two different search methods are used:

- LogisticRegressionCV
- GridSearchCV

**This code uses scikit learn 0.18. scikit learn changed packages from 0.17 to 0.18. You might need to modify the import calls if using 0.17. Upgrading to 0.18 is recommended.**

In [1]:
from sklearn import datasets
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

In [2]:
# Load in the Iris Dataset
iris = datasets.load_iris()

X_features = iris.data
Y_targets = iris.target

In [3]:
# hold out a test set to test real-world performance
# use a stratified test split for demonstration
# (although unstratified will likely be a better simulation of real-world behavior)
# stratified is recommended when dealing with rare classes
# do 10 splits to keep .10 of the data as the test set
kfold_generator = StratifiedKFold(n_splits = 10, random_state=42)
train_indices, test_indices = kfold_generator.split(X_features, Y_targets).next()

X_train = X_features[train_indices, :]
Y_train = Y_targets[train_indices]

X_test = X_features[test_indices, :]
Y_test = Y_targets[test_indices]

In [4]:
# print out the test set for demo purposes
print(X_test.shape, Y_test.shape)

((15, 4), (15,))


In [5]:
# count the iris values in the dataset
iris_values, counts = np.unique(Y_train, return_counts=True)
for value, count in zip(iris_values, counts):
    print("iris type %i, count: %i" % (value, count))

iris type 0, count: 45
iris type 1, count: 45
iris type 2, count: 45


In [6]:
# Set a search space for the C regularization parameter
# C is the inverse of lambda. A smaller C is larger regularization.

minCExp = -3
maxCExp = 4
Cs = np.logspace(minCExp, maxCExp, num = ( maxCExp - minCExp + 1))
print(Cs)

[  1.00000000e-03   1.00000000e-02   1.00000000e-01   1.00000000e+00
   1.00000000e+01   1.00000000e+02   1.00000000e+03   1.00000000e+04]


## Train using Logistic Regresion CV

My testing below suggests that the scoring functions for LogisticRegressionCV may be having a strange behavior that results in scoring that is different from GridSearchCV or from a manual cross-validation search. I recommend using GridSearchCV

In [7]:
cv_method = StratifiedKFold(n_splits=5, random_state=42)

In [8]:
# we set the Cs to be the search space of the C regularization parameter we defined above.
# we set cv to 5 to do 5 different folds of cross validation
# we set the scoring function to be log_loss, which is the cross-entropy loss
# we set the multi_class to be multinomial

logreg_cv = LogisticRegressionCV(Cs = Cs, cv=cv_method, 
                                 scoring='neg_log_loss', multi_class='multinomial', 
                                 solver='lbfgs', max_iter=10000, tol=1e-6)

In [9]:
# fit the model to the data using CV.
# note that we're not using a test set, which is typically a good thing to do
logreg_cv.fit(X_train, Y_train)

LogisticRegressionCV(Cs=array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03,   1.00000e+04]),
           class_weight=None,
           cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=False),
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=10000, multi_class='multinomial', n_jobs=1,
           penalty='l2', random_state=None, refit=True,
           scoring='neg_log_loss', solver='lbfgs', tol=1e-06, verbose=0)

*For multinomial prediction, a single C value will be chosen and shared across classes. For One Versus Rest prediction, multiple C values may be chosen.*

In [10]:
logreg_cv.C_

array([ 1000.,  1000.,  1000.])

In [11]:
# average the scores across folds
zip(Cs, np.mean(logreg_cv.scores_[0], axis=0))

[(0.001, -1.0402278943216992),
 (0.01, -0.84784093119018067),
 (0.10000000000000001, -0.65029934097579412),
 (1.0, -0.61229837068843496),
 (10.0, -0.58909076888789169),
 (100.0, -0.56296608339524501),
 (1000.0, -0.54538213334197894),
 (10000.0, -0.54740814400327786)]

*Note that it appears that scikit learn takes the negative of the log loss (which is in turn the negative of the log likelihoods) in the scoring function used by the cross validation packages.*

*Also, the loss from the LogisticRegresionCV seems to be returning some kind of unnormalized scores (maybe the raw score, not the mean negative log loss). It raises warning flags IMO.*

In [12]:
# training set performance
log_loss(Y_train, logreg_cv.predict_proba(X_train))

0.044204117903223913

In [13]:
# test set performance
log_loss(Y_test, logreg_cv.predict_proba(X_test))

0.00035985093100667984

In [14]:
# test set performance
10 * Y_test.shape[0] * log_loss(Y_test, logreg_cv.predict_proba(X_test))

0.053977639651001975

## Train using Grid Search

In [15]:
# create a cross validation method
# Here stratified k fold is used. Stratified K fold may perform better for rare targets.
# but in reality, we will not have evenly matched training and test sets, so unstratified sampling may provide
# more realistic estimates
cv_method = StratifiedKFold(n_splits=5, random_state=42)

In [16]:
logreg = sklearn.linear_model.LogisticRegression(random_state=42, max_iter = 10000, tol=1e-6, 
                                                 multi_class='multinomial', solver='lbfgs')
param_grid = {'C': Cs}

In [17]:
gs = sklearn.model_selection.GridSearchCV(logreg, param_grid=param_grid, cv = cv_method, scoring='neg_log_loss')

In [18]:
gs.fit(X_train, Y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=False),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='lbfgs',
          tol=1e-06, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03,   1.00000e+04])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [19]:
gs.cv_results_

{'mean_fit_time': array([ 0.00445914,  0.00763273,  0.01266565,  0.01918077,  0.0285008 ,
         0.03825641,  0.05675402,  0.04506173]),
 'mean_score_time': array([ 0.00073848,  0.00075407,  0.00066075,  0.00069647,  0.00066314,
         0.00065975,  0.00071096,  0.00064626]),
 'mean_test_score': array([-0.98417471, -0.65474451, -0.33890824, -0.14691963, -0.07560623,
        -0.0640402 , -0.08313179, -0.11551374]),
 'mean_train_score': array([-0.98393976, -0.65384028, -0.33617377, -0.14047801, -0.06600018,
        -0.04540575, -0.04039159, -0.03944168]),
 'param_C': masked_array(data = [0.001 0.01 0.10000000000000001 1.0 10.0 100.0 1000.0 10000.0],
              mask = [False False False False False False False False],
        fill_value = ?),
 'params': ({'C': 0.001},
  {'C': 0.01},
  {'C': 0.10000000000000001},
  {'C': 1.0},
  {'C': 10.0},
  {'C': 100.0},
  {'C': 1000.0},
  {'C': 10000.0}),
 'rank_test_score': array([8, 7, 6, 5, 2, 1, 3, 4], dtype=int32),
 'split0_test_score': arra

*Note that it appears that scikit learn takes the negative of the log loss (which is in turn the negative of the log likelihoods) in the scoring function used by the cross validation packages. Also, they return the unnormalized scores (i.e. the raw score, not the mean negative log loss).*

In [20]:
zip(Cs, gs.cv_results_['mean_test_score'])

[(0.001, -0.98417470988523692),
 (0.01, -0.65474451399536815),
 (0.10000000000000001, -0.33890824155594385),
 (1.0, -0.14691963048684026),
 (10.0, -0.075606233996511577),
 (100.0, -0.064040204703850009),
 (1000.0, -0.083131785705103461),
 (10000.0, -0.11551374162484519)]

In [21]:
gs.best_params_

{'C': 100.0}

In [22]:
best_model = gs.best_estimator_
print(best_model)

LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='lbfgs',
          tol=1e-06, verbose=0, warm_start=False)


In [23]:
# training set performance
log_loss(Y_train, best_model.predict_proba(X_train))

0.04681072157696399

In [24]:
# test set performance
log_loss(Y_test, best_model.predict_proba(X_test))

0.0017725933203487803

## Training using Logistic Regression

It isn't totally clear why the LogisticRegressionCV and GridSearchCV arrive at different results. They have slightly different best C parameters and log loss estimates.

It appears that there is simply a different randomization used for the crossvalidation fold generation in the two examples. You can see that both models use the same base model based on the replication of the results below.

In general, I would recommend using the GridSearchCV function because of it's reusability across multiple model types

In [25]:
best_C = logreg_cv.C_[0]
print("LogisticRegresionCV best C value: %f" % best_C)
logreg_model = sklearn.linear_model.LogisticRegression(random_state=42, max_iter = 10000, tol=1e-6, 
                                                 multi_class='multinomial', solver='lbfgs', C = logreg_cv.C_[0])
logreg_model.fit(X_train, Y_train)

# training set performance
print("training set performance: %f" % log_loss(Y_train, logreg_model.predict_proba(X_train)))

# test set performance
print("test set performance: %f" % log_loss(Y_test, logreg_model.predict_proba(X_test)))

LogisticRegresionCV best C value: 1000.000000
training set performance: 0.044206
test set performance: 0.000364


In [26]:
best_C = gs.best_params_['C']
print("GridSearchCV best C value: %f" % best_C)
logreg_model = sklearn.linear_model.LogisticRegression(random_state=42, max_iter = 10000, tol=1e-6, 
                                                 multi_class='multinomial', solver='lbfgs', C = gs.best_params_['C'])
logreg_model.fit(X_train, Y_train)

# training set performance
print("training set performance: %f" % log_loss(Y_train, logreg_model.predict_proba(X_train)))

# test set performance
print("test set performance: %f" % log_loss(Y_test, logreg_model.predict_proba(X_test)))

GridSearchCV best C value: 100.000000
training set performance: 0.046811
test set performance: 0.001773


## Train using Logistic Regression with Manual Cross Validation

In [27]:
cv_method = StratifiedKFold(n_splits=5, random_state=42)

In [28]:
cv_scores = np.zeros(5)
for i, (train, cv) in enumerate(cv_method.split(X_train, Y_train)):
    logreg_model = logreg = sklearn.linear_model.LogisticRegression(random_state=42, max_iter = 10000, tol=1e-6, 
                                                 multi_class='multinomial', solver='lbfgs',
                                                                   C = 1000.0)
    logreg_model.fit(X_train[train, :], Y_train[train])
    logreg_model.predict(X_train[cv, :])
    cv_scores[i] = log_loss(Y_train[cv], logreg_model.predict_proba(X_train[cv, :]))

In [29]:
-np.mean(cv_scores)

-0.083131785705103448